In [ ]:
pip install selenium

     |████████████████████████████████| 911kB 5.5MB/s 


In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
     |████████████████████████████████| 460kB 30.0MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:


from bs4 import BeautifulSoup 
import time
import pandas as pd
import numpy as np
from collections import Counter
from selenium import webdriver
import re
from PIL import Image 
from wordcloud import WordCloud, ImageColorGenerator # Image 로부터 Color 를 생성(Generate)해내는 객체입니다.
import numpy as np
from sklearn import linear_model 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
import matplotlib.pyplot as plt
import requests
from konlpy.tag import Okt
%matplotlib inline




In [ ]:


def ask_genie_any_product1(product_name1):
    

    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe',chrome_options=chrome_options)  # This is Mac version so if you use windows change the chromedriver path
    #driver =webdriver.Chrome(executable_path='/Users/paulshin/anaconda3/bin/chromedriver',chrome_options=chrome_options)


    url = 'https://shopping.naver.com/'
    driver.get(url)
    
    driver.find_element_by_xpath('//*[@id="autocompleteWrapper"]/input[1]').send_keys(product_name1) 
    driver.find_element_by_xpath('//*[@id="autocompleteWrapper"]/a[2]').click()
    url = driver.current_url
    driver.get(url)
    # 모든 상품 정보 
    data={}
    names = []
    infos = []
    like_nums = []
    prices = []
    
    url_list = []

    while 1995:
        
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        web_page = BeautifulSoup(driver.page_source, 'html.parser')
        #finding url 
        for find_url in web_page.find_all('a',{'class':'thumbnail_thumb__3Agq6'}):
            try:
                #리스트에 리뷰 추가
                url_list.append(find_url.attrs['href'])
            except: print('url error')
        
        for data in web_page.find_all('div',{'class':'basicList_info_area__17Xyo'}):
            
        # 하나씩 data에서 꺼내오기
            try:
                
                #상품이름
                name = data.find('a',{'class':'basicList_link__1MaTN'}).get_text()
                #제품설명
                info = data.find('div',{'class':'basicList_detail_box__3ta3h'}).get_text() 
                info = info.replace(' ','') #제품 설명속 공백제거 
                info = info.replace('|',' ')
                #찜 
                like_num = data.find('em',{'class':'basicList_num__1yXM9'}).get_text()
                like_num = like_num.replace(',','')
                like_num = like_num.strip()
                #가격
                price = data.find('span',{'class':'price_num__2WUXn'}).get_text()
                price = price.replace(',','')
                price = price.replace('원','')
                price= price.strip()
                
                
                #리스트에 각요소 추가
                names.append(name)
                infos.append(info)
                like_nums.append(like_num)
                prices.append(price)


                print('Process {}'.format(name))

            except:

                print('*** 다음 제품의 정보를 크롤링하는 중 에러가 발생했습니다 : {}'.format(name))  
                
                
          
            

        
        next_page = driver.find_element_by_class_name('pagination_next__1ITTf')
        next_page.click()

        time.sleep(5)

        print('============= Next Page Click ========== ')
        print('')

        # 제품 개수 제한
        if(len(names) > 100):
            driver.close()
            driver.quit()
            break
        
        
        
        
        
        
    data_frame_column_name= { "name": names, "price" : prices, "Info" : infos, "like": like_nums, 'url' : url_list}
    product_name_df = pd.DataFrame(data_frame_column_name)

    product_name_df =product_name_df[product_name_df.Info != '']
    product_name_df[['price','like']] = product_name_df[['price','like']].astype(int) 


    product_name_df = product_name_df.sort_values(by='like', ascending=False)
    product_name_df = product_name_df[:][:].reset_index()
    product_name_df = product_name_df.drop(['index'], axis=1)
    

    product_name_df.to_csv('{}_df.csv'.format(product_name1), index=False)

    info_dic = genie_show_product_keywords1(product_name_df)
    word_dic = genie_show_product_review1(product_name_df)
    
    return genie_make_wordcloud1(info_dic,word_dic, color)

In [ ]:
def genie_show_product_keywords1(df):
#-----------
    # 문자열로 like기준으로 정렬하게 되면 1 10 2 30 이런 식으로 잘못 정렬하게됨 int로 고쳐주기
    product_info = df['Info'].tolist()


    for i in range(len(product_info)):
        product_info[i] = re.sub(r'[^\w\s]',' ',product_info[i])

    # 워드 클라우드 용
    info = []
    info_dic = {}
    

    for item in product_info:
        for word in item.split(' '):
            if(word != ''):
                info.append(word)

    for word in info:
        if word not in info_dic:
            info_dic[word] = 1
        else:
            info_dic[word] += 1
            
    new_info_dic = {key : value for key, value in info_dic.items() if value > 10}
    
    return new_info_dic 

In [ ]:
def genie_show_product_review1(df):
    review = []
    for web_url in df['url']: # 각 리뷰(url)에서 리뷰본문.get_text()

        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        web = requests.get(web_url,headers=headers).content
        product_web_page = BeautifulSoup(web, 'html.parser')

        try:
            article = product_web_page.find('p',{'class':'reviewItems_text__XIsTc'})
            review.append(article.get_text())

        except:
            print('pass')


    

    articles = ' '.join(review)

    review_to_csv = pd.DataFrame(review)
    review_to_csv.to_csv('{}_review.txt'.format(product_name1))
    
    twitter = Okt()
    raw_pos_tagged = twitter.pos(articles, norm=True, stem=True) # POS Tagging

    
    
    common_list = ['하다', '있다', '되다', '이다', '돼다','매우', '않다','들다','정도','되어다','해보다','써다','쓰다','보다','자다','받다', '없다', '그렇다', '아니다', '이렇다', '그렇다', '어떻다', '광고', '크리스마스', '같다'] 
    stopwords_df = pd.read_json('stopwords-ko.json')
    stopwords= stopwords_df[0].tolist()
    word_cleaned = []
    del_list = common_list + stopwords
    # ('서울', 'Noun'),
    for word in raw_pos_tagged: #  ('서울', 'Noun'),

        if word[1] not in ["Josa", "Eomi", "Punctuation", "Foreign"]: # Foreign == ”, “ 와 같이 제외되어야할 항목들
            if (len(word[0]) != 1) & (word[0] not in del_list): # 한 글자로 이뤄진 단어들을 제외 & 원치 않는 단어들을 제외 
                word_cleaned.append(word[0])
                
    word_dic = {}

    for word in word_cleaned:
        if word not in word_dic:
            word_dic[word] = 1 # changed from "0" to "1"
        else:
            word_dic[word] += 1
            
    result = Counter(word_cleaned)
    word_dic = dict(result)        
            
    return word_dic
    

In [ ]:
def genie_make_wordcloud1(info_dic, word_dic, color) :
    key_word_logo = np.array(Image.open("lamp.jpeg"))

    function_cloud= WordCloud(font_path="C:/Users/LG/Desktop/ELAND_M.ttf", 
                              #font_path="/Users/paulshin/Library/Fonts/ELAND 나이스OTF M.otf"
                     width=2000, height=1000,
                     max_words=50, # 단어 70개만 표시
                     mask=key_word_logo,
                     background_color='black',
                     colormap = '{}'.format(color))
    
    review_word_logo = np.array(Image.open("naver.png"))

    review_cloud= WordCloud(font_path="C:/Users/LG/Desktop/ELAND_M.ttf",
                 width=2000, height=1000,
                 max_words=100, # 단어 70개만 표시
                 mask=review_word_logo,
                 background_color='black',
                 colormap = '{}'.format(color))
                     
    
    

     function_cloud.generate_from_frequencies(info_dic) 
    review_cloud.generate_from_frequencies(word_dic)
    
    plt.figure(figsize=(13,13))
    plt.imshow(function_cloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.savefig('{}_keyword_wordcloud.png'.format(product_name1))
    plt.show()
    
    
    plt.figure(figsize=(13,13))
    plt.imshow(review_cloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.savefig('{}_review_wordcloud.png'.format(product_name1))
    plt.show()

In [ ]:
product_name1 = input('지니에게 검색을 부탁하세요 : ')
color = input('지니에게 원하는 색을 알려주세요 (Blues, Reds, Greens, Oranges, Purples) : ')
ask_genie_any_product1(product_name1) 

지니에게 검색을 부탁하세요 : vans
지니에게 원하는 색을 알려주세요 (Blues, Reds, Greens, Oranges, Purples) : Greens


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


WebDriverException: ignored

In [ ]:
#-----------------------above is 1 below is two

In [ ]:
#------------------------
#testing

def ask_genie_any_product2(product_name2):
    

    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe',chrome_options=chrome_options)


    url = 'https://shopping.naver.com/'
    driver.get(url)
    
    driver.find_element_by_xpath('//*[@id="autocompleteWrapper"]/input[1]').send_keys(product_name2) # (영화이름 + 년도)로 검색
    driver.find_element_by_xpath('//*[@id="autocompleteWrapper"]/a[2]').click()
    url = driver.current_url
    driver.get(url)
    # 모든 상품 정보 
    data={}
    names = []
    infos = []
    like_nums = []
    prices = []
    
    url_list = []

    while 1995:
        
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        web_page = BeautifulSoup(driver.page_source, 'html.parser')
        #finding url 
        for find_url in web_page.find_all('a',{'class':'thumbnail_thumb__3Agq6'}):
            try:
                #리스트에 리뷰 추가
                url_list.append(find_url.attrs['href'])
            except: print('url error')
        
        for data in web_page.find_all('div',{'class':'basicList_info_area__17Xyo'}):
            
        # 하나씩 data에서 꺼내오기
            try:
                
                #상품이름
                name = data.find('a',{'class':'basicList_link__1MaTN'}).get_text()
                #제품설명
                info = data.find('div',{'class':'basicList_detail_box__3ta3h'}).get_text() 
                info = info.replace(' ','') #제품 설명속 공백제거 
                info = info.replace('|',' ')
                #찜 
                like_num = data.find('em',{'class':'basicList_num__1yXM9'}).get_text()
                like_num = like_num.replace(',','')
                like_num = like_num.strip()
                #가격
                price = data.find('span',{'class':'price_num__2WUXn'}).get_text()
                price = price.replace(',','')
                price = price.replace('원','')
                price= price.strip()
                
                
                #리스트에 각요소 추가
                names.append(name)
                infos.append(info)
                like_nums.append(like_num)
                prices.append(price)


                print('Process {}'.format(name))

            except:

                print('*** 다음 제품의 정보를 크롤링하는 중 에러가 발생했습니다 : {}'.format(name))  
                
                
          
            

        #   driver.find_element_by_class_name('pagination_next__1ITTf').click()
        next_page = driver.find_element_by_class_name('pagination_next__1ITTf')
        next_page.click()

        time.sleep(5)

        print('============= Next Page Click ========== ')
        print('')

        # 제품 개수 제한
        if(len(names) > 100):
            driver.close()
            driver.quit()
            break
        
        
        
        
        
        
    data_frame_column_name= { "name": names, "price" : prices, "Info" : infos, "like": like_nums, 'url' : url_list}
    product_name_df = pd.DataFrame(data_frame_column_name)

    product_name_df =product_name_df[product_name_df.Info != '']
    product_name_df[['price','like']] = product_name_df[['price','like']].astype(int) 


    product_name_df = product_name_df.sort_values(by='like', ascending=False)
    product_name_df = product_name_df[:][:].reset_index()
    product_name_df = product_name_df.drop(['index'], axis=1)
    

    product_name_df.to_csv('{}_df.csv'.format(product_name2), index=False)

    info_dic = genie_show_product_keywords2(product_name_df)
    word_dic = genie_show_product_review2(product_name_df)
    genie_make_wordcloud2(info_dic, word_dic, color1)
    
    return genie_make_wordcloud2(info_dic, word_dic, color1)

In [ ]:
def genie_show_product_keywords2(df):
#-----------
    # 문자열로 like기준으로 정렬하게 되면 1 10 2 30 이런 식으로 잘못 정렬하게됨 int로 고쳐주기
    product_info = df['Info'].tolist()


    for i in range(len(product_info)):
        product_info[i] = re.sub(r'[^\w\s]',' ',product_info[i])

    # 워드 클라우드 용
    info = []
    info_dic = {}
    

    for item in product_info:
        for word in item.split(' '):
            if(word != ''):
                info.append(word)

    for word in info:
        if word not in info_dic:
            info_dic[word] = 1
        else:
            info_dic[word] += 1
            
    new_info_dic = {key : value for key, value in info_dic.items() if value > 10}
    
    return new_info_dic 

In [ ]:
def genie_show_product_review2(df):
    review = []
    for web_url in df['url']: # 각 리뷰(url)에서 리뷰본문.get_text()

        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        web = requests.get(web_url,headers=headers).content
        product_web_page = BeautifulSoup(web, 'html.parser')

        try:
            article = product_web_page.find('p',{'class':'reviewItems_text__XIsTc'})
            review.append(article.get_text())

        except:
            print('pass')

    

    articles = ' '.join(review)
    
    review_to_csv = pd.DataFrame(review)
    review_to_csv.to_csv('{}_review.txt'.format(product_name2))



    twitter = Okt()
    raw_pos_tagged = twitter.pos(articles, norm=True, stem=True) # POS Tagging

    
    
    common_list = ['하다', '있다', '되다', '이다', '돼다','매우', '않다','들다','정도','되어다','해보다','써다','쓰다','보다','자다','받다', '없다', '그렇다', '아니다', '이렇다', '그렇다', '어떻다', '광고', '크리스마스', '같다'] 
    stopwords_df = pd.read_json('stopwords-ko.json')
    stopwords= stopwords_df[0].tolist()
    word_cleaned = []
    del_list = common_list + stopwords
    # ('서울', 'Noun'),
    for word in raw_pos_tagged: #  ('서울', 'Noun'),

        if word[1] not in ["Josa", "Eomi", "Punctuation", "Foreign"]: # Foreign == ”, “ 와 같이 제외되어야할 항목들
            if (len(word[0]) != 1) & (word[0] not in del_list): # 한 글자로 이뤄진 단어들을 제외 & 원치 않는 단어들을 제외 
                word_cleaned.append(word[0])
                
    word_dic = {}

    for word in word_cleaned:
        if word not in word_dic:
            word_dic[word] = 1 # changed from "0" to "1"
        else:
            word_dic[word] += 1
            
    result = Counter(word_cleaned)
    word_dic = dict(result)        
            
    return word_dic
    

In [ ]:
def genie_make_wordcloud2(info_dic, word_dic, color1) :
    key_word_logo = np.array(Image.open("lamp.jpeg"))

    function_cloud= WordCloud(font_path="C:/Windows/Fonts/malgun.ttf",
                     width=2000, height=1000,
                     max_words=50, # 단어 70개만 표시
                     mask=key_word_logo,
                     background_color='white',
                     colormap = '{}'.format(color1))
    
    review_word_logo = np.array(Image.open("naver.png"))

    review_cloud= WordCloud(font_path="C:/Windows/Fonts/malgun.ttf",
                 width=2000, height=1000,
                 max_words=100, # 단어 70개만 표시
                 mask=review_word_logo,
                 background_color='black',
                 colormap = '{}'.format(color1))
                     
    
    

     function_cloud.generate_from_frequencies(info_dic) 
    review_cloud.generate_from_frequencies(word_dic)
    
    plt.figure(figsize=(13,13))
    plt.imshow(function_cloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.savefig('{}_keyword_wordcloud.png'.format(product_name1))
    plt.show()
    
    
    plt.figure(figsize=(13,13))
    plt.imshow(review_cloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.savefig('{}_review_wordcloud.png'.format(product_name1))
    plt.show()

In [ ]:
product_name2 = input('지니에게 한번더 검색을 부탁하세요 : ')
color1 = input('지니에게 좋아하는 색갈을 골라 주세요 (Blues, Reds, Greens, Oranges, Purples) : ')
ask_genie_any_product2(product_name2) 

In [ ]:
#Finding similarity between two products

In [ ]:
#Finding the cosine similarity of the two product reviews

In [ ]:
def measure_similarity(csv_file_name1,csv_file_name2) :
    

    
    
    df_1= pd.read_csv(csv_file_name1,index_col= [0])
    df_2 = pd.read_csv(csv_file_name2,index_col= [0])
    
    df_1.dropna(inplace = True)
    df_2.dropna(inplace = True)
    
    doc1 = ''
    for line in df_1['0'] : 
        doc1 += line
    
    doc2 = ''
    for line in df_2['0'] : 
        doc2 += line
    
    corpus = [doc1, doc2]  # doc1, doc2를 합쳐 corpus list를 생성
    
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus).todense()
    
    return print("Similarity between two products : ", cosine_similarity(X[0], X[1]) )
    

In [ ]:
measure_similarity('{}_review.txt'.format(product_name1), '{}_review.txt'.format(product_name2))